# 🧠 Opdracht 2: Geheugen Toevoegen

## 🎯 Wat gaan we doen?

In dit notebook gaan we geheugen toevoegen aan onze AI! Je leert:
- Hoe message history werkt
- System, User en Assistant messages
- Een chatbot met geheugen bouwen
- Verschillende persoonlijkheden maken

## 🧠 Het Geheim: Message Lists

Tot nu toe stuurden we maar één bericht tegelijk. Voor geheugen sturen we een **lijst** van berichten!

In [ ]:
# Load environment variables from .env file
from dotenv import load_dotenv
import os

# Load the .env file from the same directory as the notebook
load_dotenv()

# Get configuration from environment variables
API_KEY = os.getenv('API_KEY', 'ollama')
BASE_URL = os.getenv('BASE_URL', 'http://localhost:11434/v1')
MODEL = os.getenv('MODEL', 'llama2')

print(f"🔗 Configuratie: {BASE_URL}")
print(f"🔑 API Key: {'*' * 8 + API_KEY[-4:] if API_KEY and len(API_KEY) > 4 else 'Niet ingesteld'}")
print(f"🤖 Model: {MODEL}")


## 🎭 De 3 Message Types

- **`system`**: Instructies voor de AI ("Je bent een piraat")
- **`user`**: Wat jij zegt
- **`assistant`**: Wat de AI zegt

Laten we dit testen:

In [4]:
# Test: AI zonder geheugen (zoals notebook 1)
def test_zonder_geheugen():
    # Eerste vraag
    response1 = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": "Mijn naam is Alex. Onthoud dit goed!"}],
        max_tokens=50
    )
    print("AI (eerste vraag):", response1.choices[0].message.content)
    
    # Tweede vraag - ZONDER de eerste vraag
    response2 = client.chat.completions.create(
        model=MODEL,
        messages=[{"role": "user", "content": "Wat is mijn naam?"}],
        max_tokens=50
    )
    print("AI (tweede vraag):", response2.choices[0].message.content)

test_zonder_geheugen()

NotFoundError: Error code: 404 - {'error': {'message': 'model "llama2" not found, try pulling it first', 'type': 'api_error', 'param': None, 'code': None}}

In [ ]:
# Test: AI MET geheugen
def test_met_geheugen():
    # Maak een message history lijst
    messages = [
        {"role": "user", "content": "Mijn naam is Alex. Onthoud dit goed!"},
    ]
    
    # Eerste API call
    response1 = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        max_tokens=50
    )
    ai_response1 = response1.choices[0].message.content
    print("AI (eerste vraag):", ai_response1)
    
    # Voeg AI antwoord toe aan geschiedenis
    messages.append({"role": "assistant", "content": ai_response1})
    
    # Voeg nieuwe gebruiker vraag toe
    messages.append({"role": "user", "content": "Wat is mijn naam?"})
    
    # Tweede API call - NU MET alle voorgeschiedenis!
    response2 = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        max_tokens=50
    )
    print("AI (tweede vraag):", response2.choices[0].message.content)
    
    print("\n📋 Volledige message history:")
    for i, msg in enumerate(messages):
        print(f"{i+1}. [{msg['role'].upper()}]: {msg['content'][:50]}...")

test_met_geheugen()

## 🤖 Chatbot Class Maken

Laten we dit netjes organiseren in een class:

In [ ]:
class SimpleChatbot:
    def __init__(self):
        self.messages = []  # Dit is ons geheugen!
        
        # Voeg een system message toe
        self.messages.append({
            "role": "system", 
            "content": "Je bent een vriendelijke AI assistent."
        })
    
    def chat(self, user_input):
        # Voeg user message toe aan geheugen
        self.messages.append({
            "role": "user",
            "content": user_input
        })
        
        # Krijg AI response
        response = client.chat.completions.create(
            model=MODEL,
            messages=self.messages,
            max_tokens=150
        )
        
        ai_response = response.choices[0].message.content
        
        # Voeg AI response toe aan geheugen
        self.messages.append({
            "role": "assistant",
            "content": ai_response
        })
        
        return ai_response
    
    def show_history(self):
        print("\n📜 Conversatie History:")
        for i, msg in enumerate(self.messages):
            role = msg['role'].upper()
            content = msg['content'][:60] + "..." if len(msg['content']) > 60 else msg['content']
            print(f"{i+1}. [{role}]: {content}")
        print()

# Test de chatbot
bot = SimpleChatbot()

print(bot.chat("Hallo! Ik ben Sarah."))
print("\n" + "-" * 40)
print(bot.chat("Wat is mijn naam?"))
print("\n" + "-" * 40)
print(bot.chat("Vertel iets over programmeren"))

bot.show_history()

## 🎭 Persoonlijkheden met System Messages

Het echte geheim zit in de **system message**. Die bepaalt hoe je AI zich gedraagt:

In [ ]:
class PersonalityBot:
    def __init__(self, personality="default"):
        self.messages = []
        self.set_personality(personality)
    
    def set_personality(self, personality):
        personalities = {
            "default": "Je bent een vriendelijke AI assistent.",
            "piraat": "Je bent een stoere piraat! Praat altijd met 'arr' en 'matey'. Vertel verhalen over de zee!",
            "professor": "Je bent een wijze professor. Leg alles uit alsof je lesgeeft. Gebruik voorbeelden.",
            "rapper": "Yo! Je bent een coole rapper. Probeer te rijmen en gebruik straattaal. Stay fresh!",
            "robot": "BEEP BOOP. JE BENT EEN ROBOT. PRAAT. ZOALS. EEN. MACHINE. GEBRUIK. ALLEEN. HOOFDLETTERS."
        }
        
        if personality in personalities:
            # Reset berichten en zet nieuwe persoonlijkheid
            self.messages = [{
                "role": "system",
                "content": personalities[personality]
            }]
            print(f"🎭 Persoonlijkheid: {personality}")
        else:
            print(f"❌ Onbekende persoonlijkheid. Kies uit: {list(personalities.keys())}")
    
    def chat(self, user_input):
        self.messages.append({"role": "user", "content": user_input})
        
        response = client.chat.completions.create(
            model=MODEL,
            messages=self.messages,
            max_tokens=150
        )
        
        ai_response = response.choices[0].message.content
        self.messages.append({"role": "assistant", "content": ai_response})
        
        return ai_response

# Test verschillende persoonlijkheden
print("🏴‍☠️ PIRAAT:")
piraat = PersonalityBot("piraat")
print(piraat.chat("Hoe gaat het met je?"))

print("\n🎓 PROFESSOR:")
prof = PersonalityBot("professor")
print(prof.chat("Wat is Python?"))

print("\n🤖 ROBOT:")
robot = PersonalityBot("robot")
print(robot.chat("Vertel over jezelf"))

## 💡 EXPERIMENTEER TIJD!

**JOUW BEURT:** Maak jouw eigen unieke persoonlijkheid!

In [ ]:
# JOUW OPDRACHT: Maak een eigen persoonlijkheid!
# Ideeën:
# - Een detective die alles mysterieus maakt
# - Een chef die alleen over eten praat
# - Een time traveler uit de middeleeuwen
# - Een alien die Earth probeert te begrijpen
# - Een superheld die altijd dramatisch doet

class MijnPersonalityBot:
    def __init__(self):
        self.messages = [{
            "role": "system",
            "content": "SCHRIJF HIER JOUW EIGEN SYSTEM MESSAGE!"  # TODO: Vervang dit!
        }]
    
    def chat(self, user_input):
        self.messages.append({"role": "user", "content": user_input})
        
        response = client.chat.completions.create(
            model=MODEL,
            messages=self.messages,
            max_tokens=150
        )
        
        ai_response = response.choices[0].message.content
        self.messages.append({"role": "assistant", "content": ai_response})
        
        return ai_response

# Test jouw persoonlijkheid hier:
# mijn_bot = MijnPersonalityBot()
# print(mijn_bot.chat("Hallo! Wie ben jij?"))

## 🧠 Memory Management

**Probleem:** Als gesprekken lang worden, wordt de message list héél groot. Dan krijg je errors!

**Oplossing:** Memory management - oude berichten weggooien:

In [ ]:
class SmartMemoryBot:
    def __init__(self, max_messages=10):
        self.max_messages = max_messages
        self.messages = [{
            "role": "system",
            "content": "Je bent een slimme AI met geheugen. Je onthoudt belangrijke dingen."
        }]
    
    def chat(self, user_input):
        # Voeg user message toe
        self.messages.append({"role": "user", "content": user_input})
        
        # Geheugen beheren VOOR API call
        self._manage_memory()
        
        # API call
        response = client.chat.completions.create(
            model=MODEL,
            messages=self.messages,
            max_tokens=150
        )
        
        ai_response = response.choices[0].message.content
        self.messages.append({"role": "assistant", "content": ai_response})
        
        return ai_response
    
    def _manage_memory(self):
        """Houd alleen de system message + laatste berichten"""
        if len(self.messages) <= self.max_messages:
            return  # Nog niet te veel berichten
        
        # Bewaar system message (eerste)
        system_msg = self.messages[0]
        
        # Houd alleen laatste berichten
        recent_messages = self.messages[-(self.max_messages-1):]
        
        # Update messages
        self.messages = [system_msg] + recent_messages
        
        print(f"🧠 Geheugen opgeruimd! Nu {len(self.messages)} berichten.")
    
    def count_messages(self):
        user_msgs = sum(1 for msg in self.messages if msg['role'] == 'user')
        ai_msgs = sum(1 for msg in self.messages if msg['role'] == 'assistant')
        system_msgs = sum(1 for msg in self.messages if msg['role'] == 'system')
        
        print(f"📊 Messages: {user_msgs} user, {ai_msgs} AI, {system_msgs} system (totaal: {len(self.messages)})")

# Test memory management
memory_bot = SmartMemoryBot(max_messages=6)  # Heel klein voor demo

vragen = [
    "Mijn naam is Kim",
    "Ik woon in Amsterdam", 
    "Mijn hobby is programmeren",
    "Wat is mijn naam?",
    "Waar woon ik?",
    "Wat is mijn hobby?",
    "Vertel alles wat je over mij weet"
]

for vraag in vragen:
    print(f"\n👤 User: {vraag}")
    antwoord = memory_bot.chat(vraag)
    print(f"🤖 AI: {antwoord}")
    memory_bot.count_messages()

## 🔄 Context Switching

Laten we experimenteren met het veranderen van persoonlijkheid tijdens een gesprek:

In [ ]:
class ContextSwitchBot:
    def __init__(self):
        self.conversation_history = []  # Bewaar alles
        self.current_personality = "default"
        self._set_system_message("default")
    
    def _set_system_message(self, personality):
        personalities = {
            "default": "Je bent een normale AI assistent.",
            "excited": "JE BENT SUPER ENTHOUSIAST! Alles is GEWELDIG! Gebruik veel uitroeptekens!!!",
            "sleepy": "Je bent heel moe... *gaaap*... probeer wakker te blijven maar je bent zo slaperig...",
            "wise": "Je bent een wijze oude man. Spreek langzaam en filosofisch. 'Hmm, interessant...'"
        }
        
        self.current_personality = personality
        self.system_message = personalities.get(personality, personalities["default"])
    
    def switch_personality(self, new_personality):
        self._set_system_message(new_personality)
        print(f"🎭 Persoonlijkheid gewijzigd naar: {new_personality}")
    
    def chat(self, user_input):
        # Voeg toe aan volledige geschiedenis
        self.conversation_history.append({"role": "user", "content": user_input})
        
        # Maak messages voor API call met huidige persoonlijkheid
        messages = [
            {"role": "system", "content": self.system_message}
        ]
        
        # Voeg recente geschiedenis toe (laatste 6 berichten)
        recent_history = self.conversation_history[-6:]
        messages.extend(recent_history)
        
        # API call
        response = client.chat.completions.create(
            model=MODEL,
            messages=messages,
            max_tokens=150
        )
        
        ai_response = response.choices[0].message.content
        self.conversation_history.append({"role": "assistant", "content": ai_response})
        
        return ai_response

# Test context switching
switch_bot = ContextSwitchBot()

print("=== NORMALE PERSOONLIJKHEID ===")
print(switch_bot.chat("Hallo! Hoe gaat het?"))

switch_bot.switch_personality("excited")
print("\n=== ENTHOUSIASTE PERSOONLIJKHEID ===")
print(switch_bot.chat("Vertel over je dag"))

switch_bot.switch_personality("sleepy")
print("\n=== SLAPERIGE PERSOONLIJKHEID ===")
print(switch_bot.chat("Ben je moe?"))

switch_bot.switch_personality("wise")
print("\n=== WIJZE PERSOONLIJKHEID ===")
print(switch_bot.chat("Wat is de zin van het leven?"))

## 📝 JOUW GROTE OPDRACHT

**CHALLENGE:** Bouw jouw eigen multi-persoonlijkheid chatbot! 

**Vereisten:**
1. Minstens 3 verschillende persoonlijkheden
2. Geheugen dat belangrijke info onthoudt
3. Een functie om persoonlijkheid te switchen
4. Memory management voor lange gesprekken

**EXTRA PUNTEN:**
- Voeg emoji's toe aan responses
- Maak een "mood" system (vrolijk, boos, verdrietig)
- Laat de AI automatisch van persoonlijkheid wisselen

In [ ]:
# JOUW EIGEN MULTI-PERSOONLIJKHEID CHATBOT HIER!
# Start met deze template:

class MijnUltimateBot:
    def __init__(self):
        self.messages = []
        self.personalities = {
            # TODO: Voeg jouw eigen persoonlijkheden toe!
            "default": "Je bent een vriendelijke assistent.",
            # "jouw_persoonlijkheid": "Beschrijving hier..."
        }
        self.current_personality = "default"
        self._reset_with_personality("default")
    
    def _reset_with_personality(self, personality):
        # TODO: Implementeer dit
        pass
    
    def switch_personality(self, personality):
        # TODO: Implementeer persoonlijkheid switchen
        pass
    
    def chat(self, user_input):
        # TODO: Implementeer chat met geheugen
        pass
    
    def show_stats(self):
        # TODO: Toon interessante statistieken
        pass

# Test jouw bot hier:
# mijn_bot = MijnUltimateBot()
# print(mijn_bot.chat("Hallo!"))

## ✅ Check Jezelf

Kun je deze vragen beantwoorden?

1. **Wat is het verschil tussen `system`, `user` en `assistant` messages?**
2. **Waarom is memory management belangrijk?**
3. **Hoe zou je een AI maken die boos wordt als je onbeleefde dingen zegt?**
4. **Wat gebeurt er als je message list te lang wordt?**

Schrijf je antwoorden hieronder:

**Mijn antwoorden:**

1. Message types: 
2. Memory management: 
3. Boze AI: 
4. Te lange lijst: 

## 🎉 Volgende Stap

Wauw! Nu heb je een AI met geheugen én persoonlijkheid gemaakt! In het volgende notebook gaan we meerdere AI's met elkaar laten praten.

**Ga naar: [03-ai-conversaties.ipynb](03-ai-conversaties.ipynb)**

---

## 💡 Tips voor Thuis

- **System messages zijn super krachtig** - experimenteer ermee!
- **Bewaar belangrijke info** in je memory management
- **Test met lange gesprekken** om je memory limits te vinden
- **Maak gekke persoonlijkheden** - dat is het leukste!